In [1]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import time
import weaviate
import requests


client = weaviate.Client("http://127.0.0.1:80")



def search(query="", limit=3):
        # Địa chỉ IP hoặc tên miền của LoadBalancer, nếu sử dụng NodePort thì dùng IP của bất kỳ node nào trong cluster.
    url = "http://127.0.0.1:81/vectorize"  # hoặc http://<Node-IP>:32001/vectorize

    # Nội dung text bạn muốn gửi
    text_data = {"text": query}

    # Gửi yêu cầu POST tới API
    response = requests.post(url, json=text_data)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        print("Vector:", response.json().get("vector"))
    else:
        print("Failed to get vector, status code:", response.status_code)

    vec = response.json().get("vector")

    near_vec = {"vector": vec}
    res = client \
        .query.get("Document", ["content", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()

    for document in res["data"]["Get"]["Document"]:
        print("{:.4f}: {}".format(document["_additional"]["certainty"], document["content"]))
        print("__________-")

search("nói về luật đất rừng ?", 3)

/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.6. The latest version is 4.7.1.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


Vector: [0.3218596279621124, -0.24036723375320435, -0.12861745059490204, -0.6723322868347168, 0.03802281245589256, -0.28629669547080994, 0.2969091534614563, -0.13954274356365204, 0.13946884870529175, -0.2872718572616577, -0.22699929773807526, -0.22884181141853333, 0.37531188130378723, 0.0986742302775383, -0.04184193164110184, 0.12232134491205215, 0.08738476783037186, 0.11568226665258408, -0.006501208990812302, 0.15285195410251617, -0.5228691697120667, -0.11486008763313293, 0.05836494266986847, -0.3335501551628113, 0.2769382894039154, 0.11030886322259903, 0.28815627098083496, 0.07053335011005402, 0.15201811492443085, -0.24766568839550018, -0.7796721458435059, -0.3392114043235779, -0.12059302628040314, -0.06604637205600739, 0.2559550106525421, -0.29703134298324585, 0.3353445529937744, -0.23147395253181458, 0.49280768632888794, 0.00840461254119873, 0.10773013532161713, -0.13120903074741364, 0.35503333806991577, -0.09871573001146317, 0.1599300354719162, -0.0008573262020945549, -0.225649967

ssh -R nthaiduong83:80:localhost:8003 serveo.net


In [12]:
import weaviate
import requests
from llama_index.core import PromptTemplate

def query_rag_llm(query_str, limit=3):
    # Khởi tạo client Weaviate
    client = weaviate.Client("http://127.0.0.1:80")

    # Gửi yêu cầu vector hóa câu truy vấn
    url = "http://127.0.0.1:81/vectorize"  
    text_data = {"text": query_str}
    response = requests.post(url, json=text_data)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        vec = response.json().get("vector")
    else:
        print("Failed to get vector, status code:", response.status_code)
        return None

    # Tìm kiếm trong Weaviate với vector truy vấn
    near_vec = {"vector": vec}
    res = client \
        .query.get("Document", ["content", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()

    # Tạo chuỗi ngữ cảnh từ các tài liệu trả về
    context_str = []
    for document in res["data"]["Get"]["Document"]:
        context_str.append("{:.4f}: {}".format(document["_additional"]["certainty"], document["content"]))
        print("{:.4f}: {}".format(document["_additional"]["certainty"], document["content"]))
        print("__________-")
    
    context_str = "\n".join(context_str)

    # Tạo PromptTemplate cho câu hỏi
    template = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please answer the question: {query_str}\n"
    )
    qa_template = PromptTemplate(template)
    messages = qa_template.format_messages(context_str=context_str, query_str=query_str)
    prompt = messages[0].content

    # Gửi yêu cầu POST tới API LLM
    api_url = "https://53e7be4092ef59381fcca638638883f0.serveo.net/complete"
    headers = {
        "ngrok-skip-browser-warning": "69420",  # You can use any value here
        "Content-Type": "application/json"  # Specify that the content is JSON
    }
    response = requests.post(api_url, json={"prompt": prompt}, headers=headers, verify=False)

    if response.status_code == 200:
        text_response = response.json().get("response")["text"]
        return text_response
    else:
        print("Failed to get response from LLM, status code:", response.status_code)
        return None

# Ví dụ sử dụng hàm
query_str = "nói về hoạt động lấn biển"
response = query_rag_llm(query_str)
print(response)

0.8314: Trích dẫn ở: luật đất đai 2024
Chương XIII
CHẾ ĐỘ SỬ DỤNG ĐẤT
Điều 190.Hoạt động lấn biển 
 Nội dung như sau: cảng cá, khu neo đậu tránh trú bão cho tàu cá theo quy định của pháp luật về thủy sản; đ) Khu vực cảng biển, vùng nước trước cầu cảng, vùng quay trở tàu, khu neo đậu, khu chuyển tải, khu tránh bão, vùng đón trả hoa tiêu, vùng kiểm dịch, luồng hàng hải, vùng nước để xây công trình phụ trợ khác theo quy định của pháp luật về hàng hải; e) Cửa sông và các khu vực đã được quy hoạch, sử dụng vào mục đích quốc phòng, an ninh. 4. Khu vực biển được xác định để thực hiện hoạt động lấn biển trong quy hoạch sử dụng
__________-
0.8099: Trích dẫn ở: luật đất đai 2024
Chương XIII
CHẾ ĐỘ SỬ DỤNG ĐẤT
Điều 190.Hoạt động lấn biển 
 Nội dung như sau: định để thực hiện hoạt động lấn biển trong quy hoạch sử dụng đất đã được phê duyệt và dự án đầu tư đã được cơ quan nhà nước có thẩm quyền chấp thuận, quyết định chủ trương đầu tư thì việc quản lý, sử dụng khu vực biển để thực hiện hoạt động lấ

/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '53e7be4092ef59381fcca638638883f0.serveo.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Câu trả lời là: Hoạt động lấn biển là việc sử dụng đất, mặt nước biển để tạo ra đất liền mới, bao gồm các hoạt động như xây dựng cảng cá, khu neo đậu tránh trú bão cho tàu cá, khu vực cảng biển, vùng nước trước cầu cảng, vùng quay trở tàu, khu neo đậu, khu chuyển tải, khu tránh bão, vùng đón trả hoa tiêu, vùng kiểm dịch, luồng hàng hải, vùng nước để xây dựng công trình phụ trợ khác theo quy định của pháp luật về hàng hải, cửa sông và các khu vực đã được quy hoạch, sử dụng vào mục đích quốc phòng, an ninh. 


In [1]:
import subprocess

# Define the list of libraries you need in the requirements file
libraries = [
    'fastapi',
    'uvicorn',
    'weaviate-client',
    'llama-index',
    'requests'
]

# Run pip freeze to get all installed packages and their versions
installed_packages = subprocess.check_output(['pip', 'freeze']).decode('utf-8').splitlines()

# Write the necessary libraries to the requirements.txt file
with open('requirements.txt', 'w') as f:
    for package in installed_packages:
        package_name = package.split('==')[0]
        if package_name in libraries:
            f.write(f"{package}\n")

print("Created requirements.txt with the necessary libraries.")


Created requirements.txt with the necessary libraries.
